In [ ]:
import watson_nlp

In [ ]:
# Set language code <lc> to language of text: Finnish 'fi', Danish 'da', Swedish 'sv', Norwegian Bokmål 'nb', English 'en', French 'fr',  etc
lc = 'fi'

In [ ]:
# Enter text input in selected language, same as <lc> above
# Text in Finnish
text_input = "Presidentti Sauli Niinistö tapaa sotaveteraaneja. Potilas on todella masentunut! Syyskuussa 2022 hän aloitti uuden lääkityksen."
# Text in Danish
#text_input = "Fra og med 2019 fremgår det af forsikringspolicen, at patienten med svær diabetes vil få refunderet gebyrer for offentlig transport til dialyse op til 100 € pr. rejse."
# Text in English
#text_input = "President Joe Biden is visiting beautiful Norway in October 2022."
# Text in French
#text_input = "Président Emmanuel Macron visite la belle Norvège en Octobre 2022."

In [ ]:
# Load Syntax block
# Check language support at https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/watson-nlp-block-syntax.html?context=cpdaas&audience=wdp
syntax_model = watson_nlp.load(watson_nlp.download('syntax_izumo_' + lc + '_stock'))
type(syntax_model)

In [ ]:
# Watson NLP functions include help
# Uncomment line below to get help on syntax model
#help(watson_nlp.blocks.syntax.IzumoTextProcessing.run)

In [ ]:
# Detect tokens, lemma and part-of-speech
syntax_prediction = syntax_model.run(text_input, parsers=('token', 'lemma', 'part_of_speech'))

In [ ]:
# Print the syntax result
print(syntax_prediction)

In [ ]:
# Print the syntax result in a more readable format
tokens = syntax_prediction.to_dict()['tokens']
for token in tokens:
        print(token['span']['text'] + ' --- ' + token['lemma'] + ' --- ' + token['part_of_speech'])

In [ ]:
# Load the Noun Phrase extraction block
# Check language support at https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/watson-nlp-block-noun-phrase.html?context=cpdaas&audience=wdp
noun_phrases_model = watson_nlp.load(watson_nlp.download('noun-phrases_rbr_' + lc + '_stock'))

In [ ]:
# Run the Noun Phrase model on the input text
noun_phrases = noun_phrases_model.run(text_input)
print(noun_phrases)

In [ ]:
# Load pre-trained multi-lingual BERT Entity block to extract machine-learning-based entities
# Check language support at
# https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/watson-nlp-block-entity-enhanced.html?context=cpdaas&audience=wdp
bert_entity_model = watson_nlp.load(watson_nlp.download('entity-mentions_bert_multi_stock'))

In [ ]:
# Run the machine-learning entity model on the result of the syntax model
bert_entity_mentions = bert_entity_model.run(syntax_prediction)
print(bert_entity_mentions)

In [ ]:
# Load rule-based Entity extraction block
rbr_entity_model = watson_nlp.load(watson_nlp.download('entity-mentions_rbr_' + lc + '_stock'))

In [ ]:
# Run the rule-based entity model on the input text
rbr_entity_mentions = rbr_entity_model.run(text_input)
print(rbr_entity_mentions)

In [ ]:
# Note that pre-trained Relation extraction is only available for English
# https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/watson-nlp-block-relation-extraction.html?context=cpdaas&audience=wdp
#relation_model = watson_nlp.load(watson_nlp.download('relations_transformer_en_stock'))
#relations_on_mentions = relation_model.run(syntax_prediction, mentions_prediction=bert_entity_mentions)
#print(relations_on_mentions.get_relation_pairs_by_type())

In [ ]:
# Load multi-lingual BERT Sentence Sentiment classification block
# Check language support at https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/watson-nlp-block-sentiment.html?context=cpdaas&audience=wdp
sentiment_model = watson_nlp.load(watson_nlp.download('sentiment_sentence-bert_multi_stock'))

In [ ]:
# Run the sentiment model on the result of the syntax model of each sentence
sentiment_result = sentiment_model.run_batch(syntax_prediction.get_sentence_texts(), syntax_prediction.sentences)
print(sentiment_result)

In [ ]:
# Calculate document sentiment using already calculated sentiment classification for each sentence
from watson_nlp.toolkit import predict_document_sentiment
document_sentiment = predict_document_sentiment(sentiment_result, sentiment_model.class_idxs)
print(document_sentiment)

In [ ]:
# Load the pre-trained Keywords block
# Check language support at https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/watson-nlp-block-keyword.html?context=cpdaas&audience=wdp
keywords_model = watson_nlp.load(watson_nlp.download('keywords_text-rank_' + lc + '_stock'))

In [ ]:
# Run the Keywords model on the result of the syntax model, limit returned keywords ranked by confidence to 3
keywords = keywords_model.run(syntax_prediction, noun_phrases, limit=3)
print(keywords)